In [ ]:
#import libraries
import re
import time
import requests
import pandas as pd
from lxml import etree
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.edge.options import Options
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException

# Part 1

Scrape 3 pages of reviews for a single product listing using product URL.<br /> Note: Product must have more than 1 review page. The code must be generic and should work for every product listing URL, with proper checks in place.

In [ ]:
url = 'https://www.daraz.pk/products/samsung-galaxy-a53-5g-8gb-ram-128gb-rom-i321318226-s1704447753.html?spm=a2a0e.searchlist.list.1.585854f2r2Ctih&search=1'

In [ ]:
driver = webdriver.Chrome(ChromeDriverManager().install())

names = []
rev = []
count = 0
driver.get(url)

for j in range(0,3):
    time.sleep(8)
    for product in driver.find_elements(By.CLASS_NAME, "item"):
        name = product.find_elements(By.CLASS_NAME, "middle")
        for value in name:
            name = value.text

        try:
            r = product.find_elements(By.CLASS_NAME, "content")
            for value in r:
                r = value.text
        except NoSuchElementException:
            r = 'Not Available'

        rev.append(r)
        names.append(name)

    nxt=driver.find_element(By.XPATH,'//*[@id="module_product_review"]/div/div/div[3]/div[2]/div/button[2]').click()
    time.sleep(5)

driver.close()
df = pd.DataFrame({'Name':names, 'Review Text':rev})
df

C:\Users\user\AppData\Local\Temp\ipykernel_6112\635793235.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


,Name,Review Text
0,by Nauman H.Verified Purchase,it was delivered on time and better price then...
1,by 3***6Verified Purchase,Seller 100% recommend. I received it packed an...
2,by Abdul R.Verified Purchase,"Quick Delivery, Owsum experience, very happy t..."
3,by faheem Q.Verified Purchase,"1st experience is very bad, 2nd time u send or..."
4,by 3***9Verified Purchase,received!! delivery on time👍🏻
5,by waqas Q.Verified Purchase,"Orignal, working well"
6,by Muhammad A.Verified Purchase,"Received in good Packing, exactly same as ment..."
7,by *******811Verified Purchase,"Package received well in time, but didnt have ..."
8,by Adnan H.Verified Purchase,The product price is too high.... The market p...
9,by Muhammad U.Verified Purchase,"Perfectly packaged, good quality and an overal..."


# Part 2

Apply Vader Sentiment to find the sentiment of the review, and append it in a column named "Sentiment".

You can get help of Vader sentiment from this; https://towardsdatascience.com/sentimental-analysis-using-vader-a3415fef7664

In [ ]:
import nltk
nltk.download('vader_lexicon')
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.sentiment.vader import SentimentIntensityAnalyzer

sid = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [ ]:
df['scores'] = df['Review Text'].apply(lambda review: sid.polarity_scores(review))
df['compound']  = df['scores'].apply(lambda score_dict: score_dict['compound'])
df['sentiment'] = df['compound'].apply(lambda c: 'pos' if c >=0 else 'neg')

df

,Name,Review Text,scores,compound,sentiment
0,by Nauman H.Verified Purchase,it was delivered on time and better price then...,"{'neg': 0.0, 'neu': 0.751, 'pos': 0.249, 'comp...",0.7430,pos
1,by 3***6Verified Purchase,Seller 100 recommend I received it packed and ...,"{'neg': 0.0, 'neu': 0.546, 'pos': 0.454, 'comp...",0.8074,pos
2,by Abdul R.Verified Purchase,Quick Delivery Owsum experience very happy to ...,"{'neg': 0.0, 'neu': 0.614, 'pos': 0.386, 'comp...",0.7425,pos
3,by faheem Q.Verified Purchase,1st experience is very bad 2nd time u send ori...,"{'neg': 0.21, 'neu': 0.667, 'pos': 0.123, 'com...",-0.5413,neg
4,by 3***9Verified Purchase,received delivery on time,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000,pos
5,by waqas Q.Verified Purchase,Orignal working well,"{'neg': 0.0, 'neu': 0.488, 'pos': 0.512, 'comp...",0.2732,pos
6,by Muhammad A.Verified Purchase,Received in good Packing exactly same as menti...,"{'neg': 0.0, 'neu': 0.766, 'pos': 0.234, 'comp...",0.6369,pos
7,by *******811Verified Purchase,Package received well in time but didnt have p...,"{'neg': 0.0, 'neu': 0.886, 'pos': 0.114, 'comp...",0.1406,pos
8,by Adnan H.Verified Purchase,The product price is too high The market price...,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000,pos
9,by Muhammad U.Verified Purchase,Perfectly packaged good quality and an overall...,"{'neg': 0.0, 'neu': 0.496, 'pos': 0.504, 'comp...",0.7964,pos


# Part 3

Scrape 80 items for a keyword search e.g., 'mobile covers'<br/>Note: Must use "click()" function of webElement, URL changing for pages is not allowed. The code must be generic and should work for every keyword, with proper checks in place. Bonus for also scraping country and Daraz Mall availability.

In [ ]:
stars_value = {
    'star-icon--k88DV star-0--WgmCt' : 0,
    'star-icon--k88DV star-1--Do7NZ' : 0.1,
    'star-icon--k88DV star-2--fBIsH' : 0.2,
    'star-icon--k88DV star-3--zmTQe' : 0.3,
    'star-icon--k88DV star-4--hM0en' : 0.4,
    'star-icon--k88DV star-5--hXNSC' : 0.5,
    'star-icon--k88DV star-6--ezGMb' : 0.6,
    'star-icon--k88DV star-7--UNNG4' : 0.7,
    'star-icon--k88DV star-8--lQLaV' : 0.8,
    'star-icon--k88DV star-9--yMyuX' : 0.9,
    'star-icon--k88DV star-10--UQtQk' : 1
}

Web_Data = pd.DataFrame([],columns=['Name', 'Price', 'Stars' ,'Reviews','Daraz Mall Availability', 'Country'])

product = 'mobile covers'
product = product.replace(" ","+")
daraz = 'https://www.daraz.pk/catalog/?q={}'.format(product)
print('URL : ',daraz,'\n')

no_of_product = 2

driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get(daraz)

#count = 1
index = 0
for ind in range(0,2):
    for product in driver.find_elements(By.CLASS_NAME,'gridItem--Yd0sa'):

        name = product.find_elements(By.CLASS_NAME,'title--wFj93')
        for value in name:
            name = value.text

        prize = int(product.find_element(By.CLASS_NAME,'currency--GVKjl').text[3:].replace(',',''))
        country = product.find_element(By.CLASS_NAME,'location--eh0Ro').text
        if country == 'Free Shipping':
            country = 'WorldWide'

        rating = 0
        try:
            stars = product.find_element(By.CLASS_NAME,'rating--ZI3Ol').get_attribute('innerHTML')
            for s in re.findall('(star-icon--k88DV star-.+?)"', stars):
                rating += stars_value[s]

        except NoSuchElementException:
            rating = 0

        daraz_mall = 0
        mall = product.find_element(By.CLASS_NAME,'tag--huSJx').get_attribute('innerHTML')
        if len(mall)>0:
            daraz_mall = 'Yes'
        else:
            daraz_mall = 'No'

        reviews = 0
        try:
            reviews = int(product.find_element(By.CLASS_NAME,'rating__review--ygkUy').text[1:-1])
        except NoSuchElementException:
            reviews = 0

        Web_Data.loc[index] = [name, prize, rating, reviews, daraz_mall, country]
        index += 1
    #count += 1
    if(index > 1):
        nxt=driver.find_element(By.XPATH,'//*[@id="root"]/div/div[2]/div/div/div[1]/div[3]/div/ul/li[9]')
        nxt.click()
        time.sleep(3)

driver.quit()
Web_Data

URL :  https://www.daraz.pk/catalog/?q=mobile+covers 



C:\Users\user\AppData\Local\Temp\ipykernel_6112\3956232761.py:24: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


,Name,Price,Stars,Reviews,Daraz Mall Availability,Country
0,FELMYST For Motorola Edge Plus (2020) Back Mob...,748,4.9,55,Yes,China
1,Hontinga for Realme C12 / C25 / C25S / Narzo 2...,500,4.8,57,Yes,WorldWide
2,Hontinga for Samsung A31 Case Luxury 6D Platin...,569,4.8,64,Yes,WorldWide
3,Hontinga for Vivo Y51 2020 Y53S 4G Y51S Back C...,500,4.9,107,Yes,WorldWide
4,Hontinga for Infinix Note 8I Back Cover Luxury...,500,4.7,79,Yes,WorldWide
...,...,...,...,...,...,...
75,Hontinga for Samsung Galaxy A53 5G Back Cover ...,500,5.0,28,Yes,WorldWide
76,Lenuo Luxury Original Square Liquid Silicone P...,509,4.6,72,Yes,WorldWide
77,Hontinga for Vivo S1 Case Luxury 6D Plating So...,500,4.9,70,Yes,WorldWide
78,Jizetin for OPPO F19 / OPPO A74 / OPPO A95 Bac...,500,4.8,67,Yes,China


# Part 4

<!-- Which Products are on Flash sale? Display Product name, Actual Price, Discounted Price, Discount Percentage, Rating, No. of Reviews, Top 3 reviews displayed.

Display output must be like this. Products may vary. -->

If a user wants to make a store to sell something, lets say "Mobile Covers". He/She wants to see folowing things:
1. Top 10 Sellers brand names
2. Daraz Mall Store/ Private store
3. Total "Mobile Covers" listings on his store
4. Avg Prices
5. Avg Units selling
6. Positive Seller Ratings
7. Seller Country

In [ ]:
Web_Data = pd.DataFrame([],columns=['Name', 'Daraz Mall/Private', 'Avg Price', 'Listings', 'Avg unit selling', 'Seller Ratings', 'Country', 'Followers'])

product = 'Mobile Covers'
product = product.replace(" ","+")
daraz = 'https://www.daraz.pk/catalog/?q={}'.format(product)

driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get(daraz)

counter = 0
index = 0
for m in range(0, 1):
    for prod in driver.find_elements(By.CLASS_NAME,'filter-panel__body--IT_3Q'):
        if counter == 0:
            n = prod.find_element(By.XPATH, '//*[@id="root"]/div/div[2]/div/div/div[2]/div/div[2]/div[2]/div/div[2]').click()

            for ind in range(1,11):
                name = driver.find_elements(By.XPATH,'//*[@id="root"]/div/div[2]/div/div/div[2]/div/div[2]/div[2]/div/div[1]/label[{}]/span[2]'.format(ind))
                for v in name:
                    name = v.text

                button = driver.find_element(By.XPATH, '//*[@id="root"]/div/div[2]/div/div/div[2]/div/div[2]/div[2]/div/div[1]/label[{}]/span[1]/input'.format(ind)).click()
                time.sleep(3)

                daraz_mall = 0
                mall = driver.find_element(By.XPATH,'/html/body/div[3]/div/div[2]/div/div/div[1]/div[3]/div[1]/div/div/div[2]/div[1]').get_attribute('innerHTML')
                if len(mall) > 0:
                    daraz_mall = 'Daraz Mall'
                else:
                    daraz_mall = 'Private'

                country = driver.find_element(By.XPATH,'/html/body/div[3]/div/div[2]/div/div/div[1]/div[3]/div[1]/div/div/div[2]/div[5]/span').text
                if country == 'Free Shipping':
                    country = 'Overseas'

                listings = driver.find_element(By.XPATH,'/html/body/div[3]/div/div[2]/div/div/div[1]/div[1]/div/div[1]/div/div/span[1]').text.replace(' items found for "','')


                driver.execute_script("arguments[0].click();", driver.find_element(By.XPATH,'//*[@id="root"]/div/div[2]/div/div/div[1]/div[3]/div[1]/div/div/div[2]/div[2]/a' ))
                time.sleep(3)
                avg_unit_selling = driver.find_element(By.XPATH,'/html/body/div[4]/div/div[3]/div[2]/div/div[1]/div[4]/div[1]/div/div/a[1]').text.replace(' Ratings','')
                if avg_unit_selling == 'No':
                    avg_unit_selling = '10'

                ratings = driver.find_element(By.XPATH, '/html/body/div[4]/div/div[3]/div[2]/div/div[2]/div[5]/div/div[2]/div[1]/div[2]').text.replace('%','')
                avg_price = driver.find_element(By.XPATH,'/html/body/div[4]/div/div[3]/div[2]/div/div[1]/div[7]/div/div/span').text.replace('Rs. ','')

                driver.execute_script("arguments[0].click();", driver.find_element(By.XPATH,'//*[@id="module_seller_info"]/div/div[1]/div[1]/div[2]/a[1]' ))
                time.sleep(3)
                followers = driver.find_element(By.XPATH, '/html/body/div[3]/div/div/div[2]/div/div/div/div[2]/div[1]/div/div/div/div[2]/div[2]/div').text.replace('Followers','')


                Web_Data.loc[index] = [name, daraz_mall, avg_price, listings, avg_unit_selling, ratings, country, followers]
                index += 1
                counter += 1

                driver.back()
                driver.back()
                button_undo = driver.find_element(By.XPATH, '//*[@id="root"]/div/div[2]/div/div/div[2]/div/div[2]/div[2]/div/div[1]/label[{}]/span[1]/input'.format(ind)).click()
                time.sleep(3)

driver.quit()
Web_Data

C:\Users\user\AppData\Local\Temp\ipykernel_6112\2292732599.py:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


,Name,Daraz Mall/Private,Avg Price,Listings,Avg unit selling,Seller Ratings,Country,Followers
0,Hontinga,Daraz Mall,569,79325,64,93,Overseas,31513
1,UCUC,Daraz Mall,499,56176,51,93,Overseas,8107
2,FELMYST,Daraz Mall,"1,199",55054,57,92,China,2026
3,Jizetin,Daraz Mall,575,57419,65,94,China,4574
4,LENUO,Daraz Mall,545,4701,267,91,Overseas,15072
5,Lereach,Daraz Mall,840,2894,36,89,Overseas,2741
6,SGP Hybrid,Daraz Mall,433,270,10,86,Overseas,633
7,Frakin,Daraz Mall,200,3,232,89,Pakistan,25855
8,,Private,999,7,2,88,China,6050
9,,Private,299,25,11,88,Pakistan,166
